# Import Necessary Libraries

In [19]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", 500)
pd.set_option("display.float_format", lambda x: '%.4f' % x)

# Import Dataset

In [20]:
online_retail = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/datasets/online_retail_II.xlsx", sheet_name="Year 2009-2010")
df = online_retail.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.9500,13085.0000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.7500,13085.0000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.7500,13085.0000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.1000,13085.0000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.2500,13085.0000,United Kingdom


# Data Preprocessing

In [21]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [22]:
df.dropna(inplace=True)

In [23]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [24]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,417534.0000,12.7588,101.2204,-9360.0000,2.0000,4.0000,12.0000,19152.0000
Price,417534.0000,3.8875,71.1318,0.0000,1.2500,1.9500,3.7500,25111.0900
Customer ID,417534.0000,15360.6455,1680.8113,12346.0000,13983.0000,15311.0000,16799.0000,18287.0000


In [25]:
df = df[~df["Invoice"].str.contains("C",na=False)]

In [26]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,407695.0000,13.5867,96.8422,1.0000,2.0000,5.0000,12.0000,19152.0000
Price,407695.0000,3.2942,34.7567,0.0000,1.2500,1.9500,3.7500,10953.5000
Customer ID,407695.0000,15368.5041,1679.7957,12346.0000,13997.0000,15321.0000,16812.0000,18287.0000


In [27]:
df = df[df["Quantity"] > 0]

# General Information About to Dataset



In [28]:
def check_df(dataframe,head=5):
  print("################################ Head ################################\n")
  print(dataframe.head(head))
  print("################################ Tail ################################\n")
  print(dataframe.tail(head))
  print("################################ Shape ################################\n")
  print(dataframe.shape)
  print("################################ Types ################################\n")
  print(dataframe.dtypes)
  print("################################ NA ################################\n")
  print(dataframe.isnull().sum())
  print("################################ Qurtiles ################################\n")
  print(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

################################ Head ################################

  Invoice StockCode                          Description  Quantity         InvoiceDate  Price  Customer ID         Country
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12 2009-12-01 07:45:00 6.9500   13085.0000  United Kingdom
1  489434    79323P                   PINK CHERRY LIGHTS        12 2009-12-01 07:45:00 6.7500   13085.0000  United Kingdom
2  489434    79323W                  WHITE CHERRY LIGHTS        12 2009-12-01 07:45:00 6.7500   13085.0000  United Kingdom
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48 2009-12-01 07:45:00 2.1000   13085.0000  United Kingdom
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24 2009-12-01 07:45:00 1.2500   13085.0000  United Kingdom
################################ Tail ################################

       Invoice StockCode                         Description  Quantity         InvoiceDate  Price  Customer ID        

# Calculate TotalPrice for any quantity

In [29]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [30]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.9500,13085.0000,United Kingdom,83.4000
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.7500,13085.0000,United Kingdom,81.0000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.7500,13085.0000,United Kingdom,81.0000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.1000,13085.0000,United Kingdom,100.8000
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.2500,13085.0000,United Kingdom,30.0000


# Calculate CLTV

In [37]:
cltv_c = df.groupby(["Customer ID"]).agg({'Invoice': lambda Invoice: Invoice.nunique(),
                                 'Quantity': lambda Quantity: Quantity.sum(),
                                 'TotalPrice': lambda TotalPrice: TotalPrice.sum()})
cltv_c.columns = ["total_transaction", "total_unit", "total_price"]

In [38]:
cltv_c.head()

,total_transaction,total_unit,total_price
Customer ID,,,
12346.0000,11,70,372.8600
12347.0000,2,828,1323.3200
12348.0000,1,373,222.1600
12349.0000,3,993,2671.1400
12351.0000,1,261,300.9300


# Calculate Average Order Value

average_order_value = total_price / total_transaction

In [39]:
cltv_c["average_order_value"] = cltv_c["total_price"] / cltv_c["total_transaction"]

In [41]:
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value
Customer ID,,,,
12346.0000,11,70,372.8600,33.8964
12347.0000,2,828,1323.3200,661.6600
12348.0000,1,373,222.1600,222.1600
12349.0000,3,993,2671.1400,890.3800
12351.0000,1,261,300.9300,300.9300


# Calculate Purchase Frequency

purchase_frequency = total_transaction / total_number_of_customers

In [45]:
cltv_c["purchase_frequency"] = cltv_c["total_transaction"] / cltv_c.shape[0]

In [46]:
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency
Customer ID,,,,,
12346.0000,11,70,372.8600,33.8964,0.0025
12347.0000,2,828,1323.3200,661.6600,0.0005
12348.0000,1,373,222.1600,222.1600,0.0002
12349.0000,3,993,2671.1400,890.3800,0.0007
12351.0000,1,261,300.9300,300.9300,0.0002


# Calculate Repeate Rate and Churn Rate

repeate_rate = number_of_customers_making_multiple_purchases / total_number_of_customers

churn_rate = 1 - repeat_rate

In [52]:
repeat_rate = cltv_c[cltv_c["total_transaction"] > 1].shape[0] / cltv_c.shape[0]
churn_rate = 1 - repeat_rate

In [53]:
churn_rate

0.3293926750115902

# Calculate Profit Margin

profit_margin = total_price * constant_value

constant_value = 0.10 ==> is a fixed value and is usually determined by the company

In [56]:
cltv_c["profit_margin"] = cltv_c["total_price"] * 0.10

In [57]:
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin
Customer ID,,,,,,
12346.0000,11,70,372.8600,33.8964,0.0025,37.2860
12347.0000,2,828,1323.3200,661.6600,0.0005,132.3320
12348.0000,1,373,222.1600,222.1600,0.0002,22.2160
12349.0000,3,993,2671.1400,890.3800,0.0007,267.1140
12351.0000,1,261,300.9300,300.9300,0.0002,30.0930


# Calculate Customer Value

customer_value = average_order_value * putchase_frequency

In [58]:
cltv_c["customer_value"] = cltv_c["average_order_value"] * cltv_c["purchase_frequency"]

In [59]:
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value
Customer ID,,,,,,,
12346.0000,11,70,372.8600,33.8964,0.0025,37.2860,0.0864
12347.0000,2,828,1323.3200,661.6600,0.0005,132.3320,0.3068
12348.0000,1,373,222.1600,222.1600,0.0002,22.2160,0.0515
12349.0000,3,993,2671.1400,890.3800,0.0007,267.1140,0.6192
12351.0000,1,261,300.9300,300.9300,0.0002,30.0930,0.0698


# Calculate Customer Lifetime Value (CLTV)

cltv = (customer_value / churn_rate) * profit_margin

In [60]:
cltv_c["cltv"] = (cltv_c["customer_value"] / churn_rate) * cltv_c["profit_margin"]

In [61]:
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
12346.0000,11,70,372.8600,33.8964,0.0025,37.2860,0.0864,9.7836
12347.0000,2,828,1323.3200,661.6600,0.0005,132.3320,0.3068,123.2355
12348.0000,1,373,222.1600,222.1600,0.0002,22.2160,0.0515,3.4733
12349.0000,3,993,2671.1400,890.3800,0.0007,267.1140,0.6192,502.1104
12351.0000,1,261,300.9300,300.9300,0.0002,30.0930,0.0698,6.3729


In [62]:
cltv_c.sort_values(by="cltv", ascending=False).head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
18102.0000,89,124216,349164.3500,3923.1949,0.0206,34916.4350,80.9375,8579573.7728
14646.0000,78,170342,248396.5000,3184.5705,0.0181,24839.6500,57.5792,4342070.4583
14156.0000,102,108107,196566.7400,1927.1249,0.0236,19656.6740,45.5648,2719105.0862
14911.0000,205,69722,152147.5700,742.1833,0.0475,15214.7570,35.2683,1629055.8098
13694.0000,94,125893,131443.1900,1398.3318,0.0218,13144.3190,30.4690,1215855.8900


In [65]:
cltv_c.describe().T

,count,mean,std,min,25%,50%,75%,max
total_transaction,4314.0000,4.4541,8.1687,1.0000,1.0000,2.0000,5.0000,205.0000
total_unit,4314.0000,1284.0111,6458.4521,1.0000,158.0000,382.0000,995.2500,220600.0000
total_price,4314.0000,2047.2887,8912.5232,0.0000,307.9500,705.5500,1722.8025,349164.3500
average_order_value,4314.0000,378.1472,492.5172,0.0000,181.9563,286.9460,423.5337,11880.8400
purchase_frequency,4314.0000,0.0010,0.0019,0.0002,0.0002,0.0005,0.0012,0.0475
profit_margin,4314.0000,204.7289,891.2523,0.0000,30.7950,70.5550,172.2802,34916.4350
customer_value,4314.0000,0.4746,2.0660,0.0000,0.0714,0.1635,0.3994,80.9375
cltv,4314.0000,5883.6065,156068.6359,0.0000,6.6737,35.0317,208.8704,8579573.7728


# Creation of Segmentation

In [66]:
cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels=["D", "C", "B", "A"])

In [67]:
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv,segment
Customer ID,,,,,,,,,
12346.0000,11,70,372.8600,33.8964,0.0025,37.2860,0.0864,9.7836,C
12347.0000,2,828,1323.3200,661.6600,0.0005,132.3320,0.3068,123.2355,B
12348.0000,1,373,222.1600,222.1600,0.0002,22.2160,0.0515,3.4733,D
12349.0000,3,993,2671.1400,890.3800,0.0007,267.1140,0.6192,502.1104,A
12351.0000,1,261,300.9300,300.9300,0.0002,30.0930,0.0698,6.3729,D


In [68]:
cltv_c.sort_values(by="cltv", ascending=False).head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv,segment
Customer ID,,,,,,,,,
18102.0000,89,124216,349164.3500,3923.1949,0.0206,34916.4350,80.9375,8579573.7728,A
14646.0000,78,170342,248396.5000,3184.5705,0.0181,24839.6500,57.5792,4342070.4583,A
14156.0000,102,108107,196566.7400,1927.1249,0.0236,19656.6740,45.5648,2719105.0862,A
14911.0000,205,69722,152147.5700,742.1833,0.0475,15214.7570,35.2683,1629055.8098,A
13694.0000,94,125893,131443.1900,1398.3318,0.0218,13144.3190,30.4690,1215855.8900,A


In [69]:
cltv_c.groupby("segment").agg({"count", "mean", "sum"})

total_transaction              total_unit                total_price                    average_order_value                   purchase_frequency              profit_margin                   customer_value                       cltv                    
                     mean count    sum       mean count      sum        mean count          sum                mean count         sum               mean count    sum          mean count         sum           mean count       sum       mean count           sum
segment                                                                                                                                                                                                                                                            
D                  1.2289  1079   1326   109.0046  1079   117616    178.1883  1079  192265.1300            157.3730  1079 169805.4655             0.0003  1079 0.3074       17.8188  1079  19226.5130         0.0413  1079   44.5677     2.6407  1079     2849.3279
C                  2.0037  1078   2160   283.0566  1078   305135    475.8965  1078  513016.4530            294.4549  1078 317422.3551             0.0005  1078 0.5007       47.5897  1078  51301.6453         0.1103  1078  118.9190    16.8685  1078    18184.2634
B                  3.7690  1078   4063   680.1586  1078   733211   1131.3592  1078 1219605.2000            389.7283  1078 420127.1237             0.0009  1078 0.9418      113.1359  1078 121960.5200         0.2623  1078  282.7087    96.0574  1078   103549.8633
A                 10.8119  1079  11666  4062.3373  1079  4383262   6401.4055  1079 6907116.4910            670.9659  1079 723972.2235             0.0025  1079 2.7042      640.1405  1079 690711.6491         1.4839  1079 1601.0933 23408.0584  1079 25257295.0412

# Export All Results to CSV

In [70]:
cltv_c.to_csv("cltv_c.csv")

# Functionalization of the All Process

In [71]:
def create_cltv_c(dataframe,profit=0.10,csv=False):
  # Data Preprocessing
  dataframe = dataframe[~dataframe["Invoice"].str.contains("C",na=False)]
  dataframe = dataframe[(dataframe["Quantity"] > 0)]
  dataframe.dropna(inplace=True)
  dataframe["TotalPrice"] = dataframe["Price"] * dataframe["Quantity"]
  cltv_c = dataframe.groupby(["Customer ID"]).agg({'Invoice': lambda Invoice: Invoice.nunique(),
                                                   'Quantity': lambda Quantity: Quantity.sum(),
                                                   'TotalPrice': lambda TotalPrice: TotalPrice.sum()})
  cltv_c.columns = ["total_transaction","total_unit","total_price"]

  # Calculate Average Order Value: average_order_value = total_price / total_transaction
  cltv_c["average_order_value"] = cltv_c["total_price"] / cltv_c["total_transaction"]
  # Calculate Purchase Frequency: purchase_frequency = total_transaction / total_number_of_customers
  cltv_c["purchase_frequency"] = cltv_c["total_transaction"] / cltv_c.shape[0]
  # Calculate Repeat Rate and Churn Rate: repeat_rate = number_of_customers_making_multiple_purchases / total_number_of_customers
  # churn_rate = 1 - repeat_rate
  repeat_rate = cltv_c[cltv_c.total_transaction > 1].shape[0] / cltv_c.shape[0]
  churn_rate = 1 - repeat_rate
  # Calculate Profit Margine: profit_margine = total_price * 0.10  ==> 0.10 is a fixed value and is usually determined by the company
  cltv_c["profit_margine"] = cltv_c["total_price"] * profit
  # Calculate Customer Value: customer_value = average_order_value * purchase_frequency
  cltv_c["customer_value"] = cltv_c["average_order_value"] * cltv_c["purchase_frequency"]
  # Calculate Customer LifTime Value: cltv = (customer_value / churn_rate) * profit_margine
  cltv_c["cltv"] = (cltv_c["customer_value"] / churn_rate) * cltv_c["profit_margine"]
  # Creation of Segments
  cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels=["D","C","B","A"])

  # Export All Results to CSV File
  if csv:
    cltv_c.to_csv("cltv_c.csv")

  return cltv_c

In [72]:
df = online_retail.copy()
create_cltv_c(df, csv=True)

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margine,customer_value,cltv,segment
Customer ID,,,,,,,,,
12346.0000,11,70,372.8600,33.8964,0.0025,37.2860,0.0864,9.7836,C
12347.0000,2,828,1323.3200,661.6600,0.0005,132.3320,0.3068,123.2355,B
12348.0000,1,373,222.1600,222.1600,0.0002,22.2160,0.0515,3.4733,D
12349.0000,3,993,2671.1400,890.3800,0.0007,267.1140,0.6192,502.1104,A
12351.0000,1,261,300.9300,300.9300,0.0002,30.0930,0.0698,6.3729,D
12352.0000,2,188,343.8000,171.9000,0.0005,34.3800,0.0797,8.3180,C
12353.0000,1,192,317.7600,317.7600,0.0002,31.7760,0.0737,7.1057,C
12355.0000,1,303,488.2100,488.2100,0.0002,48.8210,0.1132,16.7733,C
12356.0000,3,1826,3562.2500,1187.4167,0.0007,356.2250,0.8257,893.0067,A
